In [6]:
import os
import msprime, pyslim

# SLiM

SLiM stands for ["Selection on Linked Mutations"](https://messerlab.org/slim/),
written by Ben Haller and Phillip Messer.
It is [now in version 3.2.1](https://groups.google.com/forum/#!topic/slim-announce/ImGve146p6Q).
The manual is very good, and probably describes everything you'd want to know,
but there are published papers about [SLiM v2](https://academic.oup.com/mbe/article/34/1/230/2670194),
[SLiM v3](https://academic.oup.com/mbe/advance-article/doi/10.1093/molbev/msy228/5229931),
using [tree sequence recording](https://onlinelibrary.wiley.com/doi/abs/10.1111/1755-0998.12968),
and a [tutorial for SLiM v3](https://academic.oup.com/mbe/advance-article/doi/10.1093/molbev/msy237/5258474).

It was initially designed to simulate genomes under selection,
but has matured into a good ecological simulator itself (in v3).
We'll focus on this, but you still need to know the basics of how to set up genomes.

Simulation hierarchy:

- `sim` : the simulation
    - `p1`, `p2`, ... : the "subpopulations"
        - `p1.individuals` : the individuals in subpopulation `p1`
            - `ind.genomes` : the two genomes of individual `ind`
                - `genome.mutations` : the "mutations" carried by `genome`

Most things are vectorized, so for instance `p1.individuals.sex` will get you a vector of sexes,
equivalent to doing something like `for (ind in p1.individuals) { ... ind.sex ... }`.

## Running scripts

You should use the GUI, if you can, but in this notebook I'll run things on the command line.
You can `cat` a script directly to `slim`,
and therefore, you can run things through python, with the `os.system( )` call.
Here's the function we'll use to do this.

In [83]:
def slim_script(script, name, quiet=False):
    logfile = name + ".log"
    out = os.system("echo '" + script + "' | slim > " + logfile + " 2>&1")
    if not quiet:
        with open(logfile, "r") as log:
            print(log.read())
    if out != 0:
        print("An error occurred.")
    return out, logfile

### A first recipe

Here's the first recipe from the SLiM manual.
It uses the default demography: a randomly-mating Wright-Fisher population of fixed size.

In [84]:
basic_WF = """
// set up a simple neutral simulation
initialize()
{
    // set the overall mutation rate
    initializeMutationRate(1e-7);

    // m1 mutation type: neutral
    initializeMutationType("m1", 0.5, "f", 0.0);

    // g1 genomic element type: uses m1 for all mutations
    initializeGenomicElementType("g1", m1, 1.0);

    // uniform chromosome of length 100 kb
    initializeGenomicElement(g1, 0, 99999);
   
    // uniform recombination along the chromosome
    initializeRecombinationRate(1e-8);
}

// create a population of 500 individuals
1 {
    sim.addSubpop("p1", 500);
}

// run to generation 10000
10000 {
    sim.simulationFinished();
}
"""

out, logfile = slim_script(basic_WF, "basic_WF")

// Initial random seed:
1724109863656

// RunInitializeCallbacks():
initializeMutationRate(1e-07);
initializeMutationType(1, 0.5, "f", 0);
initializeGenomicElementType(1, m1, 1);
initializeGenomicElement(g1, 0, 99999);
initializeRecombinationRate(1e-08);

// Starting run at generation <start>:
1 




**Exercise:** Put this in the SLiM GUI (if you have a Mac).
1. Run it.
2. Click on the "help" button and browse the functions.
3. Option-click on a function to pop up the help for it.
4. Change the mutation type to be *beneficial*, like so:
```
    initializeMutationType("m1", 0.5, "f", 0.01);
```
and watch what happens.

## More general (non-Wright-Fisher) models

There are two general types: "Wright-Fisher" and "non-Wright-Fisher".
(Wright-Fisher is a special case of non-Wright-Fisher, but it came first, thus the name.)
We want to have real demography, so we'll stick to "nonWF" models.

### The SLiM Life Cycle

The most important thing to understand is the life cycle. 
The life cycle for Wright-Fisher models is a bit different (see the manual),
but for nonWF models it is:

1. Generation of 
    offspring

    1. Call `reproduction()` callbacks for individuals
    2. Generate the offspring
    3. Suppress/modify children with `modifyChild()` callbacks
    
2. Execution of `early()` events

3. Fitness value recalculation with `fitness()` callbacks

4. Viability/survival selection

5. Removal of fixed mutations

6. Execution of `late()` events

7. Generation count increment, individual age increments


### The parts of a recipe

Each *script block* in a SLiM recipe
is prefaced with some code that says *when* it happens, and to *whom*.
This might take the form:
```
[id] [gen1 [: gen2]] [early()] { ... }
```
The square brackets demarcate optional things.
The pieces are:

- `id` : a name for this script block (often omitted)
- `gen1` : the first time step to execute this code in
- `gen2` : the last time step to execute this code in
- `early()` : when in the life cycle this happens; can actually be one of 
    `initialize()`, `early()`, `late()`, `fitness()`, `interaction()`,
    `mateChoice()`, `modifyChild()`, `recombination()`, or `reproduction()`
    (defaults to `early()`).

These have somewhat different meanings: for instance,
an `early()` script block happens once per time step,
and a `fitness()` script block is evaluated once *per individual* each time step.
Also, many of these can be restricted to a subset of indivduals.

For instance:
```
early() {
    // stuff here happens every early()
}
```
or,
```

10:20 reproduction(p1) {
    // stuff here happens every time step from 10 to 20 (inclusive)
    // and only to individuals in subpopulation p1
}
```

### A nonWF model

Here's an example nonWF model.
The new things are that

1. We need to explicitly manage reproduction (with a `reproduction()` callback).
2. We put in some output, that reports what the current population size is every time step.

In [85]:
basic_nonWF = """
initialize()
{
    // since the model will be non-Wright-Fisher, we need to say so
    initializeSLiMModelType("nonWF");
    // genome is the same as above
    initializeMutationRate(1e-7);
    initializeMutationType("m1", 0.5, "f", 0.0);
    initializeGenomicElementType("g1", m1, 1.0);
    initializeGenomicElement(g1, 0, 99999);
    initializeRecombinationRate(1e-8);    
}

reproduction() {
    subpop.addCrossed(individual, subpop.sampleIndividuals(1));
}

1 early() {
    sim.addSubpop("p1", 10);
}

1: {
    catn(sim.generation + " : population size : " + p1.individualCount);
}

10 {
    sim.simulationFinished();
}
"""
out, logfile = slim_script(basic_nonWF, "basic_nonWF", quiet=False)


// Initial random seed:
1724139868712

// RunInitializeCallbacks():
initializeSLiMModelType(modelType = 'nonWF');
initializeMutationRate(1e-07);
initializeMutationType(1, 0.5, "f", 0);
initializeGenomicElementType(1, m1, 1);
initializeGenomicElement(g1, 0, 99999);
initializeRecombinationRate(1e-08);

// Starting run at generation <start>:
1 

1 : population size : 10
2 : population size : 20
3 : population size : 40
4 : population size : 80
5 : population size : 160
6 : population size : 320
7 : population size : 640
8 : population size : 1280
9 : population size : 2560
10 : population size : 5120



### SLiM's Fitness

Uh-oh, I see where *that* is going.
We need population regulation!
The easiest way is by the `fitness` property of individuals,
which is actually the probability of survival until the next time step.

Concretely, we'll fix a "carrying capacity" $K$,
and say that the probability of survival, per individual,
when there are $N$ individuals, is $K/N$.

As before, every individual produces one offspring
with a randomly chosen mate
every time step.

**Exercise:** What is the stable population size of this model?
Predict before you run it.

In [147]:
basic_nonWF = """
initialize()
{
    // since the model will be non-Wright-Fisher, we need to say so
    initializeSLiMModelType("nonWF");
    // genome is the same as above
    initializeMutationRate(1e-7);
    initializeMutationType("m1", 0.5, "f", 0.0);
    initializeGenomicElementType("g1", m1, 1.0);
    initializeGenomicElement(g1, 0, 99999);
    initializeRecombinationRate(1e-8);    

    // this will be the carrying capacity
    defineConstant("K", 1000);
}

reproduction() {
    subpop.addCrossed(individual, subpop.sampleIndividuals(1));
}

1 early() {
    sim.addSubpop("p1", 1000);
}

early() {
    p1.fitnessScaling = K / p1.individualCount;
}

1: early() {
    catn(sim.generation + " : population size:" + p1.individualCount);
}

100 {
    sim.simulationFinished();
}
"""
out, logfile = slim_script(basic_nonWF, "basic_nonWF")


// Initial random seed:
1732462290088

// RunInitializeCallbacks():
initializeSLiMModelType(modelType = 'nonWF');
initializeMutationRate(1e-07);
initializeMutationType(1, 0.5, "f", 0);
initializeGenomicElementType(1, m1, 1);
initializeGenomicElement(g1, 0, 99999);
initializeRecombinationRate(1e-08);

// Starting run at generation <start>:
1 

1 : population size:1000 0
2 : population size:2000 0
3 : population size:1940 0
4 : population size:2026 241
5 : population size:1966 251
6 : population size:1994 261
7 : population size:2054 257
8 : population size:1996 256
9 : population size:1860 240
10 : population size:1928 258
11 : population size:2014 243
12 : population size:2038 249
13 : population size:1990 235
14 : population size:1972 232
15 : population size:2072 255
16 : population size:1982 251
17 : population size:2020 239
18 : population size:1970 240
19 : population size:1954 234
20 : population size:1928 251
21 : population size:2006 247
22 : population size:1928 222
23 : popul

**Exercise:** Add to the output another entry: how many individuals are older than 2 time steps.
(Code like it's R;
`p1.individuals` gets you a vector of individuals; 
look in the help under Individual to see how to get their ages.)

## Subpopulations: a stage-structured model

Let's modify the above to resemble our *stage-structured* model from before.
We'll keep track of our stages with *subpopulations*:
subpopulation `p1` will be the juveniles, and `p2` the adults.

1. Individuals are *juvenile* for one year, with 90% mortality rate.
2. If they survive, they become *adults*, with 80% yearly mortality,
3. and a mean birth rate of 10 per individual.

This will grow exponentially, of course, so don't run it for too long.

In [144]:
stage_structured_recipe = """
initialize() {
    initializeSLiMModelType("nonWF");
    initializeMutationRate(1e-7);
    initializeMutationType("m1", 0.5, "f", 0.0);
    initializeGenomicElementType("g1", m1, 1.0);
    initializeGenomicElement(g1, 0, 99999);
    initializeRecombinationRate(1e-8);    

    // this will be the carrying capacity
    defineConstant("K", 100);
}

// the reproduction callback only applies to p2 (adults)
reproduction(p2) {
    num_offspring = rpois(1, 10);
    // catn("Producing " + num_offspring + " babies!");
    for (k in seqLen(num_offspring)) {
        // add new individuals to p1 (juveniles)
        p1.addCrossed(individual, subpop.sampleIndividuals(1));
    }
}

1 early() {
    // initial population
    sim.addSubpop("p1", 0);
    sim.addSubpop("p2", 10);
}


early() {
    // survival probabilities
    p1.fitnessScaling = 0.1;
    p2.fitnessScaling = 0.5;
}

late() {
    // promote all surviving juveniles to adults
    p2.takeMigrants(p1.individuals);
}

1: early() {
    cat(sim.generation + " : juveniles : " + p1.individualCount);
    catn(" : adults : " + p2.individualCount);
}

10 {
    sim.simulationFinished();
}
"""
out, logfile = slim_script(stage_structured_recipe, "stage_structured_recipe")


// Initial random seed:
1727190489285

// RunInitializeCallbacks():
initializeSLiMModelType(modelType = 'nonWF');
initializeMutationRate(1e-07);
initializeMutationType(1, 0.5, "f", 0);
initializeGenomicElementType(1, m1, 1);
initializeGenomicElement(g1, 0, 99999);
initializeRecombinationRate(1e-08);

// Starting run at generation <start>:
1 

1 : juveniles : 0 : adults : 10
2 : juveniles : 56 : adults : 6
3 : juveniles : 35 : adults : 4
4 : juveniles : 62 : adults : 7
5 : juveniles : 81 : adults : 7
6 : juveniles : 149 : adults : 14
7 : juveniles : 227 : adults : 23
8 : juveniles : 323 : adults : 31
9 : juveniles : 408 : adults : 40
10 : juveniles : 601 : adults : 65



**Exercise:**  Add *fecundity* population regulation to this model:
suppose that the mean number of offspring that an individual has
when there are $N$ adults is $10/(1 + N/K)$, where $K$ is the carrying capacity.
Predict where equilibrium will be, then run it for 1000 generations to see.

### Homework

Read sections 1-4 of the SLiM Manual.
Add *both* beneficial and deleterious mutations to a SLiM recipe of your choice.
(You should be able to tell this is working if you get fitness values both above and below 1.)